In [183]:
from config import sparqlTerms, mig_ns, sparql_mig_test, sparql_mig_simple, sparql_mig_dev
from SPARQLWrapper import JSON, SPARQLWrapper
import re, os, json, requests

In [218]:
class Query:
    def __init__(self, strlen, delim):
        self.len = strlen
        self.delim = delim
        self.sparql = SPARQLWrapper(sparql_mig_dev)
        

    def getTypebyLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?type (count(?type) as ?count) (sample(?s) as ?resource) where {?s dcterm:type ?type {select ?s ?value WHERE {?s dcterm:created ?value. filter(contains(str(?value), str('%s')) && STRLEN(str(?value)) = %s)}}} Group by ?type" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['type']['value'] != '':
                self.out.append(triple['type']['value'] + " \t " + triple['count']['value'] + "\t" + triple['resource']['value'] + "\n") 
                
    def getDateLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?value (count(?value) as ?count) (sample(?s) as ?resource) WHERE {?s dcterm:created ?value. filter(contains(str(?value), str('%s')) && (STRLEN(str(?value)) = %s))} GROUP BY ?value" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['value']['value'] != '':
                self.out.append(triple['value']['value'] + " \t " + triple['count']['value'] + "\t" + triple['resource']['value'] + "\n")    
                
    def SampleDateLen(self):
        self.sparql.setMethod("POST")
        self.sparql.setReturnFormat(JSON)
        self.out = []
        query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?value ?resource WHERE {?resource dcterm:created ?value. filter(contains(str(?value), str('%s')) && (STRLEN(str(?value)) = %s))} LIMIT 1" % (self.delim, self.len)
        self.sparql.setQuery(query)
        r  = self.sparql.query().convert()['results']['bindings']
        for triple in r:
            if triple['value']['value'] != '':
                self.out.append(triple['value']['value'] + " \t " + triple['resource']['value'] + "\n")    

In [219]:
def main():
    for i in range(1, 31):
        for j in [";", ":", "-", "/", ".", ",", " "]:
            dat = Query(i, j)
            with open("Dates/len " + str(i) + " dates.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.getDateLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")
            with open("Types/len " + str(i) + " types.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.getTypebyLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")
            with open("Sample/len " + str(i) + " sample.tsv" , "a") as file:
                print ("len is: " + str(i), "Delimiter is: " + str(j))
                dd = dat.SampleDateLen()
                file.writelines(j + "\n")
                file.writelines(dat.out)
                file.writelines("-----------------------------------------------------------" + "\n")

In [220]:
if __name__ == "__main__":
	main()

len is: 1 deliminator is: ;
len is: 1 deliminator is: ;
len is: 1 deliminator is: ;
len is: 1 deliminator is: :
len is: 1 deliminator is: :
len is: 1 deliminator is: :
len is: 1 deliminator is: -
len is: 1 deliminator is: -
len is: 1 deliminator is: -
len is: 1 deliminator is: /
len is: 1 deliminator is: /
len is: 1 deliminator is: /
len is: 1 deliminator is: .
len is: 1 deliminator is: .
len is: 1 deliminator is: .
len is: 1 deliminator is: ,
len is: 1 deliminator is: ,
len is: 1 deliminator is: ,
len is: 1 deliminator is:  
len is: 1 deliminator is:  
len is: 1 deliminator is:  
len is: 2 deliminator is: ;
len is: 2 deliminator is: ;
len is: 2 deliminator is: ;
len is: 2 deliminator is: :
len is: 2 deliminator is: :
len is: 2 deliminator is: :
len is: 2 deliminator is: -
len is: 2 deliminator is: -
len is: 2 deliminator is: -
len is: 2 deliminator is: /
len is: 2 deliminator is: /
len is: 2 deliminator is: /
len is: 2 deliminator is: .
len is: 2 deliminator is: .
len is: 2 deliminato

len is: 14 deliminator is: ,
len is: 14 deliminator is:  
len is: 14 deliminator is:  
len is: 14 deliminator is:  
len is: 15 deliminator is: ;
len is: 15 deliminator is: ;
len is: 15 deliminator is: ;
len is: 15 deliminator is: :
len is: 15 deliminator is: :
len is: 15 deliminator is: :
len is: 15 deliminator is: -
len is: 15 deliminator is: -
len is: 15 deliminator is: -
len is: 15 deliminator is: /
len is: 15 deliminator is: /
len is: 15 deliminator is: /
len is: 15 deliminator is: .
len is: 15 deliminator is: .
len is: 15 deliminator is: .
len is: 15 deliminator is: ,
len is: 15 deliminator is: ,
len is: 15 deliminator is: ,
len is: 15 deliminator is:  
len is: 15 deliminator is:  
len is: 15 deliminator is:  
len is: 16 deliminator is: ;
len is: 16 deliminator is: ;
len is: 16 deliminator is: ;
len is: 16 deliminator is: :
len is: 16 deliminator is: :
len is: 16 deliminator is: :
len is: 16 deliminator is: -
len is: 16 deliminator is: -
len is: 16 deliminator is: -
len is: 16 del

len is: 28 deliminator is: -
len is: 28 deliminator is: -
len is: 28 deliminator is: -
len is: 28 deliminator is: /
len is: 28 deliminator is: /
len is: 28 deliminator is: /
len is: 28 deliminator is: .
len is: 28 deliminator is: .
len is: 28 deliminator is: .
len is: 28 deliminator is: ,
len is: 28 deliminator is: ,
len is: 28 deliminator is: ,
len is: 28 deliminator is:  
len is: 28 deliminator is:  
len is: 28 deliminator is:  
len is: 29 deliminator is: ;
len is: 29 deliminator is: ;
len is: 29 deliminator is: ;
len is: 29 deliminator is: :
len is: 29 deliminator is: :
len is: 29 deliminator is: :
len is: 29 deliminator is: -
len is: 29 deliminator is: -
len is: 29 deliminator is: -
len is: 29 deliminator is: /
len is: 29 deliminator is: /
len is: 29 deliminator is: /
len is: 29 deliminator is: .
len is: 29 deliminator is: .
len is: 29 deliminator is: .
len is: 29 deliminator is: ,
len is: 29 deliminator is: ,
len is: 29 deliminator is: ,
len is: 29 deliminator is:  
len is: 29 del

In [ ]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
sparql.setQuery('prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT ?s ?value WHERE {?s dcterm:created ?value.}')
r  = sparql.query().convert()['results']['bindings']
for triple in r:
    if triple['value']['value'] != '':
        year = triple['value']['value']
        delim = ["-",
                 "/",
                ":",
                ";"]
        for delim1 in delim:
            if delim1 in triple['value']['value']:
                data = triple['value']['value'].split(delim1)
                if len(data) == 2:
                    data.extend(["01"])
                elif len(data) == 1:
    lowB = 6
highB = 12                data.extend(["01", "01"])
                year = data[0]
                month = data[1]
                day = data[2]
        print (year)

In [228]:
sparql = SPARQLWrapper(sparql_mig_dev)
sparql.setMethod("POST")
sparql.setReturnFormat(JSON)
for i in [";", ":", "-", "/", ".", ",", " "]:
    query = "prefix dcterm: <http://purl.org/dc/terms/> prefix info: <info:fedora/fedora-system:def/model#> prefix xsd: <http://www.w3.org/2001/XMLSchema#> SELECT (count(?s) as ?count) WHERE {?s dcterm:created ?value. filter(contains(str(?value),  str('%s')))}" % (i)
    sparql.setQuery(query)
    r  = sparql.query().convert()['results']['bindings']
    for triple in r:
        if triple['count']['value'] != '':
            print (triple['count']['value'])

2
2
228
12309
5
85
233
